# SymPauli_expect

Expectation Value of a Pauli Sum is as the Trace of PauliMatrix * DensityMatrix

$ PauliSum = \sum_{i} \ p_i \ P_i $

$ DensityMatrix = \sum_{j k} \ l_j r_k \ | L_j > < R_k | $

Thus

$ ExpectationValue = Tr(\ \sum_{i j k} \ p_j l_j r_k \ P_i | L_j > < R_k | \ ) $

While $ P_i $ is a PauliOperator (matrix) hence $ P_i | L_j > $ is becoming a new vector, it then perform the outer product with $ < R_k | $

However, for finding the Trace, all you need to do is to sum the diagonal components of the matrix.

Therefore everything outside of diagonal path can be ignored.

Which leads to:

$ ExpectationValue = \sum_{i j k} \ p_j l_j r_k \ P_i | L_j > < R_k | \ [\ iff \ P_i | L_j > == R_k \ ] $

In [170]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [171]:
P = [0,1,2,3]
L = [0,1,0,1]
R = [0,1,1,0]

# First find out bitflip Pauli
bitflips = [ (p&1)^(p>>1) for p in P]
bitflips

# Then do P_i | L_j > to make new vector
PL = [p^l for p,l in zip(bitflips, L)]
PL

if any( pl^r for pl,r in zip(PL,R) ):
    print(f"diagonal > False")
else:
    print(f"diagonal > True")


[0, 1, 1, 0]

[0, 0, 1, 1]

diagonal > False


In [172]:
# Now we verify this by actually computing the matrix representation
import numpy as np
import numexpr as ne

P = np.array([0,1,2,3], dtype='uint8')
L = np.array([0,1,0,1], dtype='uint8')
R = np.array([0,1,1,0], dtype='uint8')

# First find out bitflip Pauli
bitflips = (P&1) ^ (P>>1)
bitflips

PL = bitflips ^ L
PL

# Since this a 4qubit system, lets expand the vector
ref = (2**np.arange(4))[::-1]
ref

PLvec = np.zeros(2**4, dtype='uint8')
PLval = ref[PL.astype(bool)]
PLval
PLval = PLval.sum()
PLval
PLvec[PLval] = 1
PLvec

Rvec = np.zeros(2**4, dtype='uint8')
Rval = ref[R.astype(bool)]
Rval
Rval = Rval.sum()
Rval
Rvec[Rval] = 1
Rvec

PLRmatrix = np.outer(PLvec, Rvec)
PLRmatrix

np.trace(PLRmatrix)

array([0, 1, 1, 0], dtype=uint8)

array([0, 0, 1, 1], dtype=uint8)

array([8, 4, 2, 1], dtype=int32)

array([2, 1], dtype=int32)

3

...

array([4, 2], dtype=int32)

6

array([0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=uint8)

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], dtype=uint8)

0

# This proves the theory. So all you need to do is stack 3 for loops together.

Actually you can do 2 for loops.

Since the last step is to find if PL == R, we can do a hashtable check to accomplish this goal.

If there do no exist exact same state as PL in R set, automatically means that for every r in R we obtain zero.

In [173]:
# assume that you have already done encoding IXYZ into 0123
pauli_list = dict() # { binary pauli as tuple : coefficient }
left_state = dict() # { state as tuple : coefficient }
right_state = dict() # { state as tuple : coefficient }
expect_val = 0

# This is how you compute the coefficient for PL
# [ I X Y Z]
from operator import itemgetter as itg
from operator import mul
from functools import reduce
pl_coef = [None, None, [1j, -1j], [1, -1]]

for P, cP in pauli_list:
    #print(f"{P, cP = }")
    bitflips = [ (p&1)^(p>>1) for p in P ]

    for L, cL in left_state:
        #print(f"{L, cL = }")
        PL = tuple(p^l for p,l in zip(bitflips, L))

        # This also check whether there is a pl==r
        # At the sametime we can obtain the coefficient
        cR = right_state.get(PL, None)
        if cR is not None:
            expect_val +=  cP * cL * cR * reduce(mul, (pl_coef[p][l] for p,l in zip(P,L) if p>1), 1)
        #else:
        #    print(PL, 'unfound')

print(f"{expect_val = }")

expect_val = 0


# Benchmarking

In [174]:
%%time

# Benchmark for chain multiplication
import numpy as np

size = 100_000
n_qubits = 12
np.random.seed = 100

testP = np.random.randint(0, 4, (size, n_qubits), dtype='uint8')
#testC = np.ones(size)
testC = np.random.normal(0.5, 1, size)

experiment_data = []
for i, (coef, paulis) in enumerate(zip(testC, testP)):
    experiment_data.append([tuple(p for p in paulis), coef])
experiment_data = tuple(experiment_data)
#print(experiment_data[:10])

CPU times: total: 297 ms
Wall time: 299 ms


In [175]:
n_states = 32

denominator = 1/np.sqrt(2**n_qubits)
denominator

right_state = np.random.randint(0, 2, (n_states, n_qubits), dtype='uint8').tolist()
left_state = np.random.randint(0, 2, (n_states, n_qubits), dtype='uint8').tolist()

pauli_list = experiment_data
left_state = tuple(
    (tuple(s), denominator)
    for s in left_state
    )
right_state = tuple(
    (tuple(s), denominator)
    for s in right_state
    )

count = 0
for row in left_state:
    print(row)
    count += 1
    if count > 10:
        break

right_state = dict(right_state)
left_state = dict(left_state)
pauli_list = dict(pauli_list)

0.015625

((1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1), 0.015625)
((0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0), 0.015625)
((0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0), 0.015625)
((0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1), 0.015625)
((0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0), 0.015625)
((1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0), 0.015625)
((1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0), 0.015625)
((1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0), 0.015625)
((1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1), 0.015625)
((1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0), 0.015625)
((0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0), 0.015625)


# Naive Trace Speed

In [ ]:
%%time

from openfermion import QubitOperator as QO
from openfermion import get_sparse_operator as sparse

control_data = QO()
for i,(paulis, coef) in enumerate(zip(testP, testC)):
    term = tuple( (i, chr(87+p)) for i,p in enumerate(paulis) if p )
    control_data += QO(term, coef)
#print(control_data[:10])

Ham = sparse(control_data)

In [ ]:
experiment_data

In [ ]:
left_state_np = np.array(list(left_state.keys()), dtype='uint8')
right_state_np = np.array(list(right_state.keys()), dtype='uint8')

In [ ]:
lvec = np.zeros(2**n_qubits)
for l, cl in left_state.items():
    l = np.array(l, dtype='uint8')
    lvec[to_binary(l)] += cl
lvec
lvec.nonzero()

In [ ]:
rvec = np.zeros(2**n_qubits)
for l, cl in right_state.items():
    l = np.array(l, dtype='uint8')
    rvec[to_binary(l)] += cl
rvec
rvec.nonzero()

In [ ]:
%%time
Ham.dot(np.outer(lvec, rvec)).trace

In [ ]:
%%time
np.outer(lvec.T, rvec)

In [ ]:
%%time
np.trace(Ham @ np.outer(lvec, rvec))

In [ ]:
%%time
np.outer(lvec, rvec)

In [ ]:
%%time
lvec @ Ham @ rvec

In [ ]:
testP[:2]
print(experiment_data[0])

## PurePython Speed

In [176]:
%%time
valid_bitflips = set( tuple(l^r for l,r in zip(L,R)) for L in left_state for R in right_state )
print(len(valid_bitflips))
count = 0
for row in valid_bitflips:
    print(row)
    count += 1
    if count > 5:
        break

906
(0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0)
(0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1)
(0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1)
(1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1)
(0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0)
(0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1)
CPU times: total: 15.6 ms
Wall time: 28 ms


In [177]:
%%time

# assume that you have already done encoding IXYZ into 0123
#pauli_list = dict() # { binary pauli as tuple : coefficient }
#left_state = dict() # { state as tuple : coefficient }
#right_state = dict() # { state as tuple : coefficient }
expect_val = 0
miss = 0

# This is how you compute the coefficient for PL
# [ I X Y Z]
from operator import itemgetter as itg
from operator import mul
from functools import reduce
pl_coef = [None, None, [1j, -1j], [1, -1]]

for P, cP in pauli_list.items():
    #print(f"{P, cP = }")
    bitflips = tuple( (p&1)^(p>>1) for p in P )
    if bitflips in valid_bitflips:

        for L, cL in left_state.items():
            #print(f"{L, cL = }")
            PL = tuple(p^l for p,l in zip(bitflips, L))

            # This also check whether there is a pl==r
            # At the sametime we can obtain the coefficient
            cR = right_state.get(PL, None)
            if cR is not None:
                expect_val +=  cP * cL * cR * reduce(mul, (pl_coef[p][l] for p,l in zip(P,L) if p>1), 1)
            else:
                miss += 1
            #    print(PL, 'unfound')

print(f"{expect_val = }")
print(f"{miss = }")

expect_val = (-0.007711647429912435-0.0006391002909370575j)
miss = 685797
CPU times: total: 5.3 s
Wall time: 5.3 s


## Binary Numpy Speed

In [178]:
def to_binary(arr):
    bin_ref = (2**np.arange(arr.shape[-1], dtype=np.uint64))[::-1]
    return np.dot(arr, bin_ref)

def get_valid_bitflips(left, right):
    """
    Examples:
        >>> get_valid_bitflips([[3]],[[5]])  # [[0,1,1]], [[1,0,1]]
        6
    """
    out = set()
    for l in left:
        out.update(l^right)
    return out

def pauli_to_bitflips(P):
    """
    Examples:
        >>> ppp = np.array([1,2,3], dtype='uint8')
        >>> pauli_to_bitflips(ppp)
        6
    """
    return to_binary((P&1) ^ (P>>1))


In [179]:
%%time

debug = 0

left_state_np = np.array(list(left_state.keys()), dtype='uint8')
left_state_keys = to_binary(left_state_np)  # [int, int, int, int, ...]
left_state_bin = dict(zip(left_state_keys, left_state.items()))

right_state_np = np.array(list(right_state.keys()), dtype='uint8')
right_state_keys = to_binary(right_state_np)  # [int, int, int, int, ...]
right_state_bin = dict(zip(right_state_keys, right_state.values()))

if debug:
    count = 0
    for row in left_state_bin.items():
        print(row)
        count += 1
        if count > 3:
            break
    count = 0
    for row in right_state_bin.items():
        print(row)
        count += 1
        if count > 3:
            break

# 1. XOR(left_state, right_state) to find the appropriate bitflips to make them matches
valid_bitflips = get_valid_bitflips(left_state_keys, right_state_keys)
print(len(valid_bitflips))
if debug:
    count = 0
    for row in valid_bitflips:
        print(row, bin(row))
        count += 1
        if count > 5:
            break

# 2. Translate Pauli's bitflip behaviour to binary representation
pauli_list_np = np.array(list(pauli_list.keys()), dtype='uint8')
bitflips_list = to_binary((pauli_list_np&1) ^ (pauli_list_np>>1))
if debug:
    count = 0
    for row in bitflips_list:
        print(row, bin(row))
        count += 1
        if count > 5:
            break

# 4. if and only if valid_bitflips
valid_pauli_list = { p: (bitflips, cp) for bitflips, (p, cp) in zip(bitflips_list, pauli_list.items()) if bitflips in valid_bitflips }

if debug:
    count = 0
    for row in valid_pauli_list.items():
        print(row)
        count += 1
        if count > 3:
            break

906
CPU times: total: 125 ms
Wall time: 127 ms


In [180]:
%%time

expect_val = 0
miss = 0

from operator import itemgetter as itg
from operator import mul
from functools import reduce
# This is how you compute the coefficient for PL
# [ I X Y Z]
pl_coef = [None, None, [1j, -1j], [1, -1]]

for P, (binbitflips, cP) in valid_pauli_list.items():
    #print(f"{P, bitflips, cP = }")
    for binL, (L, cL) in left_state_bin.items():
        #print(f"{L, cL = }")
        binPL = binL ^ binbitflips
        # This also check whether there is a pl==r
        # At the sametime we can obtain the coefficient
        cR = right_state_bin.get(binPL, None)   # right_state_bin = { binR: (cR) }
        if cR:
            expect_val +=  cP * cL * cR * reduce(mul, (pl_coef[p][l] for p,l in zip(P,L) if p>1), 1)
        else:
            miss += 1

print(f"{expect_val = }")
print(f"{missrate = :.2%}")

expect_val = (-0.007711647429912435-0.0006391002909370575j)
missrate = 0.00%
CPU times: total: 719 ms
Wall time: 718 ms


## Binary Numpy BatchCoef Speed

In [181]:
%%time

#Ver.BatchCoef
expect_val = 0
miss = total = 0

from operator import itemgetter as itg
from operator import mul
from functools import reduce, lru_cache

# This is how you compute the coefficient for PL
# [ I X Y Z]
pl_coef = [None, None, [1j, -1j], [1, -1]]

@lru_cache(maxsize=256)
def _PaulizalCoef(_P, _L):
    return reduce(mul, (pl_coef[p][l] for p,l in zip(_P,_L) if p>1), 1)

def PaulizalCoef(P, L, pt=2):
    return reduce(mul, (_PaulizalCoef(P[k:k+pt], L[k:k+pt]) for k in range(0, len(P), pt)) , 1)

for P, (binbitflips, cP) in valid_pauli_list.items():
    #print(f"{P, bitflips, cP = }")
    for binL, (L, cL) in left_state_bin.items():
        #print(f"{L, cL = }")
        # This also check whether there is a pl==r
        # At the sametime we can obtain the coefficient
        cR = right_state_bin.get(binL ^ binbitflips, None)  # right_state_bin = { binR: (cR) }
        if cR:
            expect_val += cP * cL * cR * PaulizalCoef(P,L)

print(f"{expect_val = }")

expect_val = (-0.007711647429912435-0.0006391002909370575j)
CPU times: total: 328 ms
Wall time: 329 ms


## Batch coef (?)

In [ ]:
%%time
from cProfile import run

run("""
#Ver.BatchCoef
expect_val = 0

# This is how you compute the coefficient for PL
# [ I X Y Z]
from operator import itemgetter as itg
from operator import mul
from functools import reduce
pl_coef = [None, None, [1j, -1j], [1, -1]]

@lru_cache(maxsize=None)
def _PaulizalCoef(_P, _L):
    return reduce(mul, (pl_coef[p][l] for p,l in zip(_P,_L) if p>1), 1)

def PaulizalCoef(P, L, pt=4):
    return reduce(mul, (_PaulizalCoef(P[k:k+pt], L[k:k+pt]) for k in range(0, len(P), pt)) , 1)

for P, (binbitflips, cP) in valid_pauli_list.items():
    #print(f"{P, bitflips, cP = }")

    for binL, (L, cL) in left_state_bin.items():
        #print(f"{L, cL = }")
        binPL = binL ^ binbitflips

        # This also check whether there is a pl==r
        # At the sametime we can obtain the coefficient
        cR = right_state_bin.get(binPL, None)  # right_state = { binR: (R, cR) }
        if cR is not None:
            expect_val +=  cP * cL * cR[1] * PaulizalCoef(P,L)

print(f"{expect_val = }")
""")

In [182]:
P = (2, 0, 2, 1, 3, 3, 2, 3, 3, 1, 0, 2, 3, 2)
L = (0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1)

pt = 4
for k in range(0, len(P), pt):
    end = k+pt
    p = P[k:end]
    l = L[k:end]
    print(p, l)

(2, 0, 2, 1) (0, 1, 0, 0)
(3, 3, 2, 3) (1, 0, 1, 0)
(3, 1, 0, 2) (0, 0, 1, 1)
(3, 2) (0, 1)


# BitflipBridge

In [183]:
class BitflipBridge(dict):
    
    def __missing__(self, key):
        out = self[key] = [[], []]
        return out
    

def to_binary(arr):
    bin_ref = (2**np.arange(arr.shape[-1], dtype=np.uint64))[::-1]
    return np.dot(arr, bin_ref)

def pauli_to_bitflips(P):
    """
    Examples:
        >>> ppp = np.array([1,2,3], dtype='uint8')
        >>> pauli_to_bitflips(ppp)
        6
    """
    return to_binary((P&1) ^ (P>>1))


In [184]:
%%time

from cProfile import run
from line_profiler import LineProfiler

profile = LineProfiler()

debug = 0

left_state_npbin = to_binary(np.array(list(left_state.keys()), dtype='uint8'))
left_state_dict = dict(zip(
    left_state_npbin,
    left_state.items()
))

right_state_npbin = to_binary(np.array(list(right_state.keys()), dtype='uint8'))
right_state_dict = dict(zip(
    right_state_npbin,
    right_state.items()
))

bridge = BitflipBridge()

for l, (tuplel, cl) in left_state_dict.items():
    for bitflips, (r, (tupler, cr)) in zip(l^right_state_npbin, right_state_dict.items()):
        bridge[bitflips][0].append((l, tuplel, cl, r, tupler, cr))

# 2. Translate Pauli's bitflip behaviour to binary representation
bitflips_list = pauli_to_bitflips(np.array(list(pauli_list.keys()), dtype='uint8'))
#if debug:
#    count = 0
#    for row in bitflips_list:
#        print(row, bin(row))
#        count += 1
#        if count > 5:
#            break
for bitflips, (tuplep, cp) in zip(bitflips_list, pauli_list.items()):
    if bridge[bitflips][0]:
        bridge[bitflips][1].append((tuplep, cp))

if debug:
    count = 0
    for bf, (lr, p) in bridge.items():
        print(bf)
        print(lr)
        print(p, '\n')
        count += 1
        if count > 5:
            break

del left_state_dict
del left_state_npbin
del right_state_dict
del right_state_npbin
del bitflips_list



CPU times: total: 328 ms
Wall time: 324 ms


In [185]:
%%time

#Ver.BatchCoef
expect_val = 0

from operator import itemgetter as itg
from operator import mul
from functools import reduce, lru_cache
# This is how you compute the coefficient for PL
# [ I X Y Z]
pl_coef = [None, None, [1j, -1j], [1, -1]]

@lru_cache(maxsize=256)
def _PaulizalCoef(_P, _L):
    return reduce(mul, (pl_coef[p][l] for p,l in zip(_P,_L) if p>1), 1)

def PaulizalCoef(P, L, pt=2):
    return reduce(mul, (_PaulizalCoef(P[k:k+pt], L[k:k+pt]) for k in range(0, len(P), pt)) , 1)

for bf, (LRs, Ps) in bridge.items():
    #print(f"{P, bitflips, cP = }")
    for l, tuplel, cl, r, tupler, cr in LRs:
        for tuplep, cp in Ps:
            expect_val += cp * cl * cr * PaulizalCoef(tuplep,tuplel)

print(f"{expect_val = }")

expect_val = (-0.007711647429912406-0.0006391002909369952j)
total = 0
CPU times: total: 156 ms
Wall time: 147 ms
